In [1]:
!pip install transformers trl accelerate torch bitsandbytes peft datasets -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset, concatenate_datasets

# Load your dataset
dataset = load_dataset('csv', data_files='pricelabs_finetuning.csv')

# Factor by which to repeat the dataset
replication_factor = 3  # You can change this to any number

# Create a list of the dataset repeated 'replication_factor' times
datasets_to_concatenate = [dataset['train']] * replication_factor


# Concatenate the datasets
train_test_split = concatenate_datasets(datasets_to_concatenate)




# Now split the repeated dataset into train and test sets
split_datasets = train_test_split.train_test_split(test_size=0.3)

# The training set (70% of the data)
train_dataset = split_datasets['train']

# The test set (30% of the data)
test_dataset = split_datasets['test']

FileNotFoundError: Unable to find '/content/pricelabs_finetuning.csv'

In [ ]:
test_dataset

#### Create Formatted Prompt

In the following function we'll be merging our `prompt` and `response` columns by creating the following template:

```
<s>### Instruction:
Use the provided input to create an instruction that could have been used to generate the response with an LLM.

### Input:
{input}

### Response:
{response}</s>
```

In [3]:
def create_prompt(sample):
  bos_token = "<s>"
  original_system_message = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
  system_message = "Use the provided input to create an instruction that could have been used to generate the response with an LLM."
  response = sample["prompt"].replace(original_system_message, "").replace("\n\n### Instruction\n", "").replace("\n### Response\n", "").strip()
  input = sample["response"]
  eos_token = "</s>"

  full_prompt = ""
  full_prompt += bos_token
  full_prompt += "### Instruction:"
  full_prompt += "\n" + original_system_message
  full_prompt += "\n\n### Input:"
  full_prompt += "\n" + input
  full_prompt += "\n\n### Response:"
  full_prompt += "\n" + response
  full_prompt += eos_token

  return full_prompt

### Loading the Base Model

We're going to load our model in `4bit`, with double quantization, with `bfloat16` as our compute dtype.

You'll notice we're loading the instruct-tuned model - this is because it's already adept at following tasks - we're just teaching it a new one!

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    device_map='auto',
    quantization_config=nf4_config,
    use_cache=False
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [7]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=32768, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [9]:
generate_response(
    "### Instruction:\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.### Input:\nTell me about the capital of India?\n\n### Response:", model)

"<s> \nThe capital of India is New Delhi. It is a city and a union territory of India containing the New Delhi municipality and the Parliament of India. It is located in thenorthern part of the Indian subcontinent, and is a vibrant metropolis with a rich history and diverse culture. New Delhi is known for its iconic landmarks such as the Red Fort, Qutub Minar, and Humayun's Tomb, as well as its delicious cuisine, bustling street markets, and historical monuments.</s>"

Now, we're going to prepare our model for 4bit LoRA training!

We can use these handy helper functions to achieve this goal thanks to huggingface and the peft library!



In [ ]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
  output_dir = "mistral_instruct_generation",
  # num_train_epochs=2,
  max_steps = 400, # comment out this line if you want to train in epochs
  per_device_train_batch_size = 4,
  warmup_steps = 0.03,
  logging_steps=10,
  save_strategy="epoch",
  #evaluation_strategy="epoch",
  evaluation_strategy="steps",
  eval_steps=20, # comment out this line if you want to evaluate at the end of each epoch
  learning_rate=2e-4,
  bf16=True,
  lr_scheduler_type='constant',
)

In [ ]:
from trl import SFTTrainer

max_seq_length = 2048

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt,
  args=args,
  train_dataset=train_dataset,
  eval_dataset=test_dataset
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:302: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
20,0.531600,0.582147
40,0.527400,0.582147
60,0.527700,0.582147
80,0.528500,0.582147
100,0.539300,0.582147
120,0.536100,0.582147
140,0.532500,0.582147
160,0.530200,0.582147
180,0.520800,0.582147
200,0.524500,0.582147


Checkpoint destination directory mistral_instruct_generation/checkpoint-7 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Checkpoint destination directory mistral_instruct_generation/checkpoint-14 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be upda

TrainOutput(global_step=400, training_loss=0.5323878240585327, metrics={'train_runtime': 1594.9253, 'train_samples_per_second': 1.003, 'train_steps_per_second': 0.251, 'total_flos': 1.24860005710037e+17, 'train_loss': 0.5323878240585327, 'epoch': 57.14})

In [ ]:
trainer.save_model("pricelabs_instruct_generation")

# Save Model and Push to Hub

4bit save and push coming soon!

The PR is literally in the process of being added! Check it out [here](https://github.com/TimDettmers/bitsandbytes/pull/753)!

For now, we'll save our adapters!

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install --upgrade huggingface-hub



In [ ]:
merged_model = model.merge_and_unload()

In [ ]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

In [ ]:
generate_response(
    "### Instruction:\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.### Input:\nHow to set minimum stay restrictions in Pricelabs?\n\n### Response:",
    merged_model)

'<s> ### Instruction:\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.### Input:\nHow to set minimum stay restrictions in Pricelabs?\n\n### Response:\nTo set minimum stay restrictions in Pricelabs, follow these steps:\n1. Log in to your Pricelabs account.\n2. Go to the "Properties" tab and select the property for which you want to set minimum stay restrictions.\n3. Under the "Pricing Rules" section, select the "Add Rule" button.\n4. Choose "Minimum Stay" as the rule type.\n5. Enter the number of nights that you want to set as the minimum stay for the property.\n6. Select the days of the week or specific dates on which the minimum stay should apply.\n7. Save the rule.\n\nWith these steps, you can ensure that your property is only available for short-term rentals or longer stays as per your preference.</s>'

In [ ]:
<s> \nMinimum stay restrictions can be set by specifying the minimum number of days that a user must stay in a particular location before they are allowed to check out. This can be done by going to the "Settings" menu and selecting "Minimum Stay Restrictions." From there, you can select the desired minimum stay duration and save your changes. It\'s important to note that these restrictions can only be set by the property owner or manager and may vary depending on the specific location.</s>

Object `PriceLabs` not found.


In [ ]:
generate_response(
    "### Instruction:\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.### Input:\nCan you show me how to set up minimum stay limits for my listings in PriceLabs?\n\n### Response:",
    merged_model)

"<s> ### Instruction:\nBelow is an instruction that describes a task. Write a response that appropriately completes the request.### Input:\nCan you show me how to set up minimum stay limits for my listings in PriceLabs?\n\n### Response:\nOf course! To set up minimum stay limits for your listings in PriceLabs, follow these steps:\n\n1. Log in to your PriceLabs account.\n2. Click on the listing for which you want to set minimum stay limits.\n3. In the 'Restrictions' tab, click on theEdit button' next to 'Minimum Stay'.\n4. Set the number of nights you want as the minimum stay for your listing.\n5. Click on the 'SAVE' button.\n\nOnce this is done, PriceLabs will ensure that your listing is only booked for the minimum number of nights you've specified.</s>"